### **Check if the new structure is not slowing down the code**

In [1]:
using OrdinaryDiffEq, DiffEqDevTools,  Test, BenchmarkTools

import ODEProblemLibrary: prob_ode_linear,
                          prob_ode_2Dlinear


For the previous structure :

In [18]:
@benchmark solve(prob_ode_linear, Tsit5())

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.740 μs … 540.050 μs  ┊ GC (min … max):  0.00% … 98.64%
 Time  (median):     2.300 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   2.801 μs ±  10.732 μs  ┊ GC (mean ± σ):  11.27% ±  3.09%

   ▅▅▅▅▆▆▆▇██▇▅▄▃▃▃▃▃▂▃▂▂▁▁▁▁▁▁▁▁▁▁▁  ▁▁  ▁                   ▃
  █████████████████████████████████████████▇█▇█▆▆▆▆▆▅▅▅▄▄▃▅▂▆ █
  1.74 μs      Histogram: log(frequency) by time      5.16 μs <

 Memory estimate: 4.47 KiB, allocs estimate: 36.

In [15]:
@benchmark solve(prob_ode_2Dlinear, Tsit5())

BenchmarkTools.Trial: 10000 samples with 6 evaluations.
 Range (min … max):  5.933 μs … 397.350 μs  ┊ GC (min … max): 0.00% … 94.93%
 Time  (median):     6.567 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.658 μs ±  11.116 μs  ┊ GC (mean ± σ):  5.87% ±  4.02%

  ▃▇█▇▅▅▅▄▄▄▄▄▅▅▅▅▄▂▁▁▁▁ ▁                                    ▂
  ██████████████████████████████▇██▇▇▇▆██▇▇▆▆▆▆▅▄▇▇▇▇▇▇▆▇▇▆▆▄ █
  5.93 μs      Histogram: log(frequency) by time      13.6 μs <

 Memory estimate: 14.70 KiB, allocs estimate: 110.

For the new structure :

In [17]:
@benchmark solve(prob_ode_linear, Tsit5_for_relaxation())

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.770 μs … 505.320 μs  ┊ GC (min … max):  0.00% … 97.31%
 Time  (median):     2.000 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   2.572 μs ±  10.111 μs  ┊ GC (mean ± σ):  11.65% ±  3.08%

  ▃▇███▆▅▄▄▄▅▆▅▄▂▁▂▁▁ ▁▁▁▁▁▁▁▁▁▂▂▁▂▂▁▁▁▁▁▁▁ ▁                 ▃
  ███████████████████████████████████████████▇▇█▇▇▇▇▅▅▆▆▆▅▅▄▄ █
  1.77 μs      Histogram: log(frequency) by time      4.57 μs <

 Memory estimate: 4.47 KiB, allocs estimate: 36.

In [16]:
@benchmark solve(prob_ode_2Dlinear, Tsit5_for_relaxation())

BenchmarkTools.Trial: 10000 samples with 5 evaluations.
 Range (min … max):  5.920 μs … 366.900 μs  ┊ GC (min … max): 0.00% … 95.12%
 Time  (median):     6.760 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.733 μs ±  12.003 μs  ┊ GC (mean ± σ):  5.79% ±  3.68%

  ▁▆██▇▇▇▆▅▅▅▅▅▅▅▆▅▄▂▁▂▁   ▁▁▁▁▂▁▂▁▁▁▁▁                       ▃
  ███████████████████████████████████████▇▇▆▇▆▆▅▅▆▆▆▇▆▅█▇▇▇▆▆ █
  5.92 μs      Histogram: log(frequency) by time      13.3 μs <

 Memory estimate: 14.70 KiB, allocs estimate: 110.